In [ ]:
# Khai báo các thư hiện cần thiết

In [2]:
import zipfile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

# Giải nén file tải xuống
def unzip_data(filename):
  """
  Function giải nén file.
  """
  zip_ref = zipfile.ZipFile(filename, "r")
  zip_ref.extractall()
  zip_ref.close()

In [3]:
# Khai báo kiểu tập dữ liệu train và test
IMG_SIZE = (224, 224)

def create_data_loaders(train_dir, test_dir, image_size=IMG_SIZE):
  """
 Tạo tập data train và data test BatchDataset từ train_dir và test_dir.
  """
  train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                  label_mode="categorical",
                                                                  image_size=image_size)
  
  test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                  label_mode="categorical",
                                                                  image_size=image_size)
  
  return train_data, test_data

In [4]:
# Tăng cường data với các trường hợp xoay, lật ngang, thu phóng...
data_augmentation = keras.Sequential([
  preprocessing.RandomFlip("horizontal"),
  preprocessing.RandomRotation(0.2),
  preprocessing.RandomZoom(0.2),
  preprocessing.RandomHeight(0.2),
  preprocessing.RandomWidth(0.2),
  # preprocessing.Rescaling(1./255) # keep for ResNet50V2, remove for EfficientNetB0
], name ="data_augmentation")

In [5]:
# Thiết lập input shape và khởi tạo model với các layer cần thiết
INPUT_SHAPE = (224, 224, 3)
BASE_MODEL = tf.keras.applications.EfficientNetB0(include_top=False)

def create_model(input_shape=INPUT_SHAPE, base_model=BASE_MODEL, num_classes=10):
  
  base_model.trainable = False

  # Khởi tạo input layer
  inputs = layers.Input(shape=input_shape, name="input_layer")

  # Thêm giai đoạn tăng cường data thành 1 layer
  x = data_augmentation(inputs)

  # cho model cơ sở 1 đầu vào nhưng không train
  x = base_model(x, training=False)

  # Tạo layer Pooling2D
  x = layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)

  # Đặt layer dense ở output
  outputs = layers.Dense(num_classes, activation="softmax", name="output_layer")(x)

  # khai báo model với 2 đầu input và output
  model = keras.Model(inputs, outputs)

  # Compile the model
  model.compile(loss="categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
  
  return model

16711680/16705208 [==============================] - 4s 0us/step


In [6]:
# Function để import ảnh và resize lại để nhập vào model
def load_and_prep_image(filename, img_shape=224, scale=False):
  """
  Đọc dữ liệu ảnh từ filename, decode lại và chuyển thành kích thước
  (224, 224, 3).
  """
  # Đọc ảnh vào
  img = tf.io.read_file(filename)
  # Decode lại
  img = tf.image.decode_jpeg(img)
  # Resize ảnh
  img = tf.image.resize(img, [img_shape, img_shape])
  # Rescale ảnh (get all values between 0 and 1)
  if scale:
    return img/255.
  else:
    return img

In [7]:
# Tải xuống tập data để train
import zipfile

# Tải data (10 class subset of Food101 - https://www.kaggle.com/dansbecker/food-101)

!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_all_data.zip

unzip_data("10_food_classes_all_data.zip")

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [8]:
# Tạo tensorboard callback
import datetime
def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir
  )
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback

In [9]:
# Tạo BatchDataset
train_data, test_data = create_data_loaders(train_dir="10_food_classes_all_data/train/",
                                            test_dir="10_food_classes_all_data/test/")

Found 7500 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [10]:
# Kiểm tra data train
train_data

<BatchDataset shapes: ((None, 224, 224, 3), (None, 10)), types: (tf.float32, tf.float32)>

In [11]:
# Tạo model
model_1 = create_model(num_classes=len(train_data.class_names))

# Train model
history_1_percent = model_1.fit(train_data,
                    epochs=5,
                    steps_per_epoch=len(train_data),
                    validation_data=test_data,
                    validation_steps=int(0.25 * len(test_data)), # validate for less steps
                    # Track model training logs
                    callbacks=[create_tensorboard_callback("transfer_learning", "all_data_aug")])

Saving TensorBoard log files to: transfer_learning/all_data_aug/20211113-181703
Epoch 1/5
235/235 [==============================] - 476s 2s/step - loss: 1.4580 - accuracy: 0.5497 - val_loss: 0.4722 - val_accuracy: 0.8701
Epoch 2/5
235/235 [==============================] - 380s 2s/step - loss: 0.7374 - accuracy: 0.7699 - val_loss: 0.4243 - val_accuracy: 0.8750
Epoch 3/5
235/235 [==============================] - 361s 2s/step - loss: 0.6411 - accuracy: 0.7985 - val_loss: 0.3574 - val_accuracy: 0.8980
Epoch 4/5
235/235 [==============================] - 374s 2s/step - loss: 0.5888 - accuracy: 0.8150 - val_loss: 0.3459 - val_accuracy: 0.9030
Epoch 5/5
235/235 [==============================] - 365s 2s/step - loss: 0.5609 - accuracy: 0.8285 - val_loss: 0.3403 - val_accuracy: 0.9013


In [12]:
# Save model_1
model_1.save("efficientnet_model_1_10_classes")

INFO:tensorflow:Assets written to: efficientnet_model_1_10_classes\assets
